In [209]:
## importando libraries
import pandas as pd
import numpy as np
import os
import re
import requests
from bs4 import BeautifulSoup

In [210]:
url = 'http://g1.com.br/'
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/51.0.2704.103 Safari/537.36'}



req = requests.get(url,headers= header)
html = req.text
soup = BeautifulSoup(html,'html.parser')
colecao = soup.find_all('a',class_ ="menu-item-link menu-navegacao-color-primary",href=True) #div eh texto

In [211]:
# retirar links e títulos
links = re.findall('https.*/',str(colecao))
titulo = re.findall('title.*</span>',str(colecao))

In [213]:
# limpar conteudo dos titulos
for i in range(len(titulo)):
    titulo[i] = titulo[i].replace('title">','').replace('</span>','')

In [214]:
G1 = pd.DataFrame(list(zip(titulo, links)), columns=['Titulo', 'Links'])

In [215]:
G1.head()

,Titulo,Links
0,Primeira Página,https://g1.globo.com/economia/agronegocios/
1,Agro de gente pra gente,https://g1.globo.com/economia/agronegocios/agr...
2,Globo Rural,https://g1.globo.com/economia/agronegocios/glo...
3,A Indústria-Riqueza do Brasil,https://g1.globo.com/economia/agronegocios/agr...
4,primeira página,https://g1.globo.com/carnaval/2023/


In [216]:
def retiraG1(url_principal,header): 
  ## função que entra em um link principal encontrado encima e gera os links sobre o mesmo assunto
  req = requests.get(url_principal,headers= header)
  html = req.text
  soup = BeautifulSoup(html,'html.parser')
  noticias = soup.find_all('a',class_="feed-post-link gui-color-primary gui-color-hover",href=True)
  # retirar links e títulos
  links = []
  titulo = []
  for i in range(len(noticias)):
    links.append(re.findall('https.*ghtml',str(noticias[i]))[0])
    titulo.append(re.findall('">.*</a>',str(noticias[i])))
    # limpar conteudo dos titulos
  for i in range(len(titulo)):
      titulo[i] = titulo[i][0].replace('">','').replace('</a>','')
  G1_2 = pd.DataFrame(list(zip(titulo, links)), columns=['Titulo', 'Links'])
  return(G1_2)

In [217]:
retiraG1(G1['Links'][3],header) ## filtra links correspondentes de acordo com o titulo principal

,Titulo,Links
0,"Após chuvas, lavouras de pimenta-rosa seguem a...",https://g1.globo.com/es/espirito-santo/noticia...
1,Modernização da agricultura oferece oportunida...,https://g1.globo.com/jornal-nacional/noticia/2...
2,Rendimento do trabalhador rural pode variar de...,https://g1.globo.com/jornal-nacional/noticia/2...
3,A tecnologia move o homem do campo; conheça a ...,https://g1.globo.com/economia/agronegocios/agr...
4,Vaca é arrematada por quase R$ 6 milhões em le...,https://g1.globo.com/sp/itapetininga-regiao/no...
5,Chuvas destroem plantações e afetam a safra de...,https://g1.globo.com/jornal-nacional/noticia/2...
6,O agro é a diversidade das lavouras e das técn...,https://g1.globo.com/economia/agronegocios/agr...
7,Gente do campo: mulher vive há quase 50 anos t...,https://g1.globo.com/economia/agronegocios/agr...
8,Agricultor tem opção de picape em que pode con...,https://g1.globo.com/economia/agronegocios/agr...
9,Saiba quais as diferenças entre os cafés conil...,https://g1.globo.com/mg/sul-de-minas/grao-sagr...


Retirar texto correspondente e a data

In [218]:
G1['Titulo'][12] ## pegar assuntos relacionados a bitcoin
bitcoin = retiraG1(G1['Links'][12],header)

In [219]:
bitcoin.head()

,Titulo,Links
0,"Vencedor do Nobel, Paul Krugman decreta o 'fim...",https://g1.globo.com/economia/noticia/2022/12/...
1,Bitcoin e ether dão sinais de recuperação e an...,https://g1.globo.com/economia/noticia/2022/09/...
2,"O que é a fusão do Ethereum, tida como 'maior ...",https://g1.globo.com/economia/noticia/2022/09/...
3,Bitcoin: o país onde é possível comprar quase ...,https://g1.globo.com/economia/bitcoin/noticia/...
4,'Rei do Bitcoin' é condenado por estelionato e...,https://g1.globo.com/pr/parana/noticia/2022/04...


In [220]:
def retira_texto(link_texto,header):
  # pega o link do texto e retorna o texto
  req = requests.get(link_texto,headers= header)
  html = req.text
  soup = BeautifulSoup(html,'html.parser')
  texto = soup.find_all('p', class_="content-text__container")
  data = soup.find_all('div', class_="content-publication-data__text")
  # retirar o texto
  texto_trat = []
  for i in range(len(texto)):
    try:
      texto_trat.append(re.findall('links="">.*</p>',str(texto[i]))[0])
    except:
      pass
  # Trata texto
    for i in range(len(texto_trat)):
        texto_trat[i] = texto_trat[i].replace('links="">','').replace('</p>','').strip()
  # Trata data
  data = re.findall('"datePublished">.*</time> <span',str(data))[0]
  data = data.replace('"datePublished">','').replace('</time> <span','').strip()
  return(texto_trat,data)

In [221]:
bitcoin['Titulo'][1] ## titulo do texto de busca

"Bitcoin e ether dão sinais de recuperação e analistas questionam 'fundo do poço'"

In [222]:
retira_texto(bitcoin['Links'][1],header) ## retirar o texto

(['Após testarem novas mínimas em mais de dois meses, bitcoin, ethereum e algumas das principais criptomoedas deram sinais de recuperação nesta terça, apesar do mau humor nos mercados de risco com o ajuste na política monetária para conter a escalada dos preços.',
  'Os ganhos do segmento cripto acontecem na véspera de decisão de juros pelo Federal Reserve, quando a maioria do mercado espera nova alta de 0,75 ponto na taxa, mas alguns temem uma elevação de 1 ponto percentual.',
  'Perto das 8h45 (horário de Brasília), o bitcoin tinha alta de 3,1% nas últimas 24 horas, negociado a US$ 19.305,13, e o ether, moeda digital da rede ethereum, subia 3,9%, a US$ 1.361, conforme dados do CoinGecko.',
  'O valor de mercado somado de todas as criptomoedas é de US$ 976 bilhões. Em reais, o bitcoin registrava alta de 1,37% a R$ 100.378,57, enquanto o ethereum tinha ganhos de 1,95% a R$ 7.080,52, de acordo com valores fornecidos pelo MB.',
  'Segundo André Franco, chefe de pesquisa do MB, a última o